# Corpus compilation
- text type and genres
- characteristics according to task

In [1]:
# Requests in python
import urllib3
# Regular expression library
import re
# Transform html into a tree im memory
from bs4 import BeautifulSoup
import os
# Get just the text in a html document
import justext

In [2]:
# Disable https warning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# Pass as a browser 
user_agent = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}

# Pool of boots to make requests
http = urllib3.PoolManager(10, headers=user_agent)


class Crawler:
    
    def __init__(self, corpus_path, max_files, seed_url, url_pattern):
        self.corpus_path = corpus_path # corpus address
        self.max_files = max_files # max amount of sizes
        self.seed_url = seed_url # root url
        self.url_pattern = url_pattern # Select links of interest
        self.visited_links = [] # Hash to store viseted links
        self.to_be_visited = [] # List of link
        
        # If path not exists create it
        if not os.path.exists(self.corpus_path):
            os.makedirs(self.corpus_path)
        
    def crawl(self):
        first_urls = self.get_page(self.seed_url)
        self.add_links(first_urls)
        next_link = self.get_next_link()
        
        file_counter = 1
        while next_link and file_counter < self.max_files:
            links = self.get_page(next_link)
            self.add_links(links)
            next_link = self.get_next_link()
            file_counter += 1
        
    
    def get_page(self, url):
        print("getting page {}".format(url))
        response = http.request('GET', url)

        # store text content
        paragraphs = justext.justext(response.data, justext.get_stoplist("Portuguese"))
        with open("{}/{}.txt".format(self.corpus_path, url.replace(".", "_").replace("/","-")[8:]), "w") as output_file:
            for paragraph in paragraphs:
                # Boilerplate is everthing that is not the main text
                if not paragraph.is_boilerplate:
                    output_file.write(paragraph.text)
        
        # get links
        soup = BeautifulSoup(response.data, 'html.parser')
        
        links = [link.get('href') for link in soup.findAll('a', attrs={'href': re.compile(self.url_pattern)})]
        return links

    def add_links(self, links):
        links = list(set(links))
        self.to_be_visited.extend([link for link in links if link not in self.visited_links])

    def get_next_link(self):
        next_link = self.to_be_visited.pop(0)
        self.visited_links.append(next_link)
        return next_link

In [7]:
crawler_folha_saude = Crawler("data/corpora/saude", 500, 
                             "https://www1.folha.uol.com.br/equilibrioesaude/2019/04/quase-80-dos-brasileiros-se-automedicam-diz-pesquisa-datafolha.shtml", 
                            "^https://www1\.folha\.uol\.com\.br/equilibrioesaude/\d+")

crawler_uol_saude = Crawler("data/corpora/saude", 500,
                           "https://noticias.uol.com.br/saude/ultimas-noticias/bbc/2019/04/27/a-extraordinaria-cientista-que-estudou-o-cerebro-de-einstein-e-revolucionou-a-neurociencia-moderna.htm",
                           "^https://noticias\.uol\.com\.br/saude/")

crawler_drauzilio = Crawler("data/corpora/saude", 500,
                             "https://drauziovarella.uol.com.br/hipertensao/7-mitos-e-verdades-sobre-hipertensao/",
                             "^https://drauziovarella\.uol\.com\.br/")

crawler_abril_saude = Crawler("data/corpora/saude", 500,
                             "https://saude.abril.com.br/blog/futuro-do-diabete/pre-diabetes-nao-e-uma-pre-doenca/",
                             "^https://saude\.abril\.com\.br/blog/")

crawler_folha_politica = Crawler("data/corpora/politica", 500,
                          "https://www1.folha.uol.com.br/poder/2019/04/brasil-e-governado-por-um-bando-de-maluco-diz-lula-em-entrevista-na-prisao.shtml",
                          "^https://www1\.folha\.uol\.com\.br/poder/\d+")

crawler_uol_politica = Crawler("data/corpora/politica", 500, "https://economia.uol.com.br/noticias/redacao/2019/04/25/reforma-da-previdencia-bpc-rural-abono.htm",
                              "^https://economia\.uol\.com\.br/noticias/")

crawler_paragmatismo_politica = Crawler("data/corpora/politica", 500, "https://www.pragmatismopolitico.com.br/2019/04/ataques-carlos-bolsonaro-contra-mourao-twitter.html",
                                       "^https://www\.pragmatismopolitico\.com\.br/2019/\d+")

crawler_oglobo_politica = Crawler("data/corpora/politica", 500, "https://blogs.oglobo.globo.com/lauro-jardim/post/bolsonaro-veta-campanha-do-banco-do-brasil-marcada-pela-diversidade-e-diretor-cai-veja-o-video-proibido.html",
                                 "^https://blogs\.oglobo\.globo\.com/")

crawler_carta_politica = Crawler("data/corpora/politica", 500, "https://www.cartacapital.com.br/politica/em-meio-a-ataques-de-carlos-bolsonaro-maia-nega-impeachment-de-mourao/",
                                "^https://www\.cartacapital\.com\.br/politica/")


In [9]:
# crawler_folha_saude.crawl()
# crawler_uol_saude.crawl()
# crawler_drauzilio.crawl()
# crawler_abril_saude.crawl()


# crawler_folha_politica.crawl()
crawler_uol_politica.crawl()
crawler_carta_politica.crawl()
crawler_paragmatismo_politica.crawl()
crawler_oglobo_politica.crawl()



getting page https://economia.uol.com.br/noticias/redacao/2019/04/25/reforma-da-previdencia-bpc-rural-abono.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/26/diversidade-comercial-do-bb-tem-mais-negros-do-que-media.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/24/reforma-da-previdencia-o-que-muda-fgts-aposentado.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/26/indicacao-do-relator-da-reforma-desagrada-a-lideranca-do-centrao.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/25/escolhas-para-comissao-especial-permitem-fazer-costura-com-partidos-diz-joice.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/23/oposicao-justica-sigilo-reforma-da-previdencia.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/24/ganho-com-reforma-pode-chegar-a-r-35066-bi-para-estados-em-10-anos-diz-governo.htm
getting page https://economia.uol.com.br/notici

getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/24/centrao-quer-tirar-estados-de-reforma.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/25/veja-apresentacao-do-governo-sobre-dados-que-embasam-reforma-da-previdencia.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/23/oposicao-consegue-110-assinaturas-e-quer-suspender-votacao-da-reforma.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/23/lider-do-podemos-diz-que-partido-vai-tentar-retirar-estados-da-reforma.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/23/reforma-da-previdencia-ccj.htm
getting page https://economia.uol.com.br/noticias/reuters/2019/04/24/governo-tem-que-se-empenhar-mais-pela-reforma-da-previdencia-diz-marinho.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/25/comissao-especial-da-previdencia-camara-relator-presidente.htm
getting page https://economia.uol.com.br/noticias/redacao

getting page https://economia.uol.com.br/noticias/redacao/2019/04/24/comissao-especial-camara-deputados-reforma-previdencia.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/25/oposicao-ingressa-com-mandado-no-stf-para-cancelar-sessao-da-ccj-sobre-reforma.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/23/ccj-bate-boca-danilo-gentili-reforma-da-previdencia.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/24/reforma-e-matematica-nao-ideologia.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/26/articulacao-politica-para-previdencia-comecou-a-entrar-nos-trilhos-apos-ccj.htm
getting page https://economia.uol.com.br/noticias/reuters/2019/04/25/parlamentares-do-psl-defendem-novo-regime-para-previdencia-como-poupanca-garantida-apos-reuniao-com-guedes.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/27/servidor-que-ganha-mais-de-r-10-mil-sera-o-mais-afeta

getting page https://economia.uol.com.br/noticias/redacao/2019/04/24/governo-pode-garantir-ate-36-votos-na-comissao-especial-a-favor-da-reforma.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/23/oposicao-diz-que-vai-recorrer-contra-resultado-da-ccj-da-previdencia.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/26/ex-comunista-deputado-diz-que-fez-releitura-ideologica.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/25/reforma-da-previdencia-custo-capitalizacao.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/24/reforma-da-previdencia-ccj-parlamentares.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/25/reforma-da-previdencia-economia-contribuicao.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/26/reforma-da-previdencia-pontos-polemicos-marcelo-ramos.htm
getting page https://economia.uol.com.br/noticias/bloomberg/2019/04/23/investidor-espera-atraso-e-

getting page https://economia.uol.com.br/noticias/reuters/2019/04/24/governo-tem-que-se-empenhar-mais-pela-reforma-da-previdencia-diz-marinho.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/25/comissao-especial-da-previdencia-camara-relator-presidente.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/25/reforma-da-previdencia-bpc-rural-abono.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/26/reforma-pode-gerar-43-milhoes-de-empregos-ate-2022-diz-bolsonaro.htm
getting page https://economia.uol.com.br/noticias/estadao-conteudo/2019/04/23/vitor-hugo-diz-que-instalacao-da-comissao-especial-sera-na-quinta-feira.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/24/oferta-de-r-40-milhoes-para-aprovar-reforma-e-criminosa-diz-olimpio.htm
getting page https://economia.uol.com.br/noticias/redacao/2019/04/24/em-pronunciamento-bolsonaro-diz-que-pobres-pagarao-menos-com-reforma.htm
getting page https://econo

...

getting page https://blogs.oglobo.globo.com/lauro-jardim/post/hora-da-verdade-para-paulo-guedes-e-bolsonaro.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/tse-julga-criacao-de-partido-que-recebeu-apoio-de-bolsonaro.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/irritacao-de-paulo-guedes.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/filha-de-rodrigo-maia-foi-alvo-de-ameacas-pela-internet.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/filha-de-rodrigo-maia-foi-alvo-de-ameacas-pela-internet.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/bolsonaro-e-o-onus-da-reforma.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/tag/jair-bolsonaro/5.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/bolsonaro-e-o-onus-da-reforma.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/tag/jair-bolsonaro/4.html
getting page https://blog

getting page https://blogs.oglobo.globo.com/lauro-jardim/servico/tag/jair-bolsonaro/34.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/bolsonaro-chama-kassab-para-conversar.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/bolsonaro-vai-falar-em-cadeia-nacional.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/ultima-de-maia-sobre-bolsonaro-como-sera-o-amanha.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/bolsonaro-impoe-horario-de-quartel-ministros.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/tag/ministros/
getting page https://blogs.oglobo.globo.com/lauro-jardim/tag/reforma-da-previdencia/
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/bolsonaro-vai-falar-em-cadeia-nacional.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/em-cadeia-nacional-bolsonaro-fala-da-nova-previdencia-e-elogia-rodrigo-maia.html#comments
getting page https://blogs.oglobo.g

getting page https://blogs.oglobo.globo.com/lauro-jardim/post/bolsonaro-olavo-e-mourao.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/depois-do-encontro-dos-presidentes-o-dos-vices-de-brasil-e-eua.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/perguntas-sobre-bolsonaro-que-nao-querem-calar.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/maia-facada-contribuiu-para-vitoria-de-bolsonaro.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/maia-facada-contribuiu-para-vitoria-de-bolsonaro.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/tag/jair-bolsonaro/1.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/servico/tag/jair-bolsonaro/8.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/servico/tag/jair-bolsonaro/6.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/o-desarranjo-do-governo-no-congresso.html
getting page https://blogs.oglobo.globo.com/l

getting page https://blogs.oglobo.globo.com/lauro-jardim/post/campanha-que-pede-cassacao-de-jair-bolsonaro-ja-tem-quase-300-mil-assinaturas.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/bolsonaro-quer-se-candidatar-prefeito.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/livro-sobre-educacao-sexual-que-irritou-bolsonaro-foi-comprado-pelo-minc-em-2010.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/servico/tag/jair-bolsonaro/33.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/servico/tag/jair-bolsonaro/31.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/jair-bolsonaro-e-hostilizado-em-voo-para-paris-canalha-homifobico.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/bolsonaro-quer-se-candidatar-prefeito.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/campanha-que-pede-cassacao-de-jair-bolsonaro-ja-tem-quase-300-mil-assinaturas.html#comments
getting page http

getting page https://blogs.oglobo.globo.com/lauro-jardim/post/hoje-inimigo-de-mourao-ha-dois-meses-carlos-o-homenageava.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/comunidade-judaica-representa-criminalmente-contra-ciro-gomes.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/ancine-promove-servidor-citado-em-acordao-do-tcu.html#comments
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/mudancas-na-epson-brasil.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/roberto-carlos-nas-telas-em-3d.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/ancine-promove-servidor-citado-em-acordao-do-tcu.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/3.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/post/concessionarias-de-rodovias-e-aeroportos-preparam-o-ataque.html
getting page https://blogs.oglobo.globo.com/lauro-jardim/tag/rubem-novaes/
getting page https://blogs.oglobo.globo.c